In [1]:
import os
import json
import cv2
import numpy as np

In [2]:
base_path = 'annotated/'
out_detection_path = 'detection_dataset'
out_recognition_path = 'recognition_dataset'

In [4]:
detection_annotations = []
recognition_annotations = []
for date in os.listdir(base_path):
    for clip in os.listdir(os.path.join(base_path, date)):
        with open(os.path.join(base_path, date, clip, 'output.json')) as f:
            orig_annotation = json.load(f)
        clip_meta = orig_annotation.copy()
        objects = {pig['id']: pig['frames'] for pig in orig_annotation['objects']}
        del clip_meta['objects']
        prev_bbox = {}
        cap = cv2.VideoCapture(os.path.join(base_path, date, clip, 'color.mp4'))
        for i in range(0, int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), 3):
            cap.set(1, i)
            ret, frame = cap.read()
            frame_filename = f'{date}__{clip}__{i}.jpeg'
            frame_detection_annotation = {'frame_filename': frame_filename, 'bboxes':[], **clip_meta}
            for _id, bboxes in objects.items():
                if len(bboxes) >0 and bboxes[0]['frameNumber']*3 == i:
                    prev_bbox[_id] = bboxes.pop(0)
                box = prev_bbox[_id].copy()
                box['frameNumber'] = i
                box['id'] = _id
                frame_detection_annotation['bboxes'].append(box.copy())
                
                box_coords = box['bbox']
                recognition_frame = frame[box_coords['y']:box_coords['y']+box_coords['height'],
                                          box_coords['x']:box_coords['x']+box_coords['width'] , :]
                recognition_filename = f'{date}__{clip}__{i}__{_id}.jpeg'
                cv2.imwrite(os.path.join(out_recognition_path, recognition_filename), recognition_frame)
                box['recognition_filename'] = recognition_filename
                recognition_annotations.append(box)
            cv2.imwrite(os.path.join(out_detection_path, frame_filename), frame)
            detection_annotations.append(frame_detection_annotation)

In [5]:
with open('detection_annotations.json', 'w') as outfile:
    json.dump(detection_annotations, outfile)
with open('recognition_annotations.json', 'w') as outfile:
    json.dump(recognition_annotations, outfile)

In [32]:
base_path = 'annotated/'
out_detection_path = 'detection_dataset_masked'

In [33]:
mask = cv2.imread(os.path.join(base_path, 'mask.jpg'), cv2.IMREAD_GRAYSCALE)>100
mask = np.repeat(mask[:, :, np.newaxis], 3, axis=2)
for date in os.listdir(base_path):
    for clip in os.listdir(os.path.join(base_path, date)):
        with open(os.path.join(base_path, date, clip, 'output.json')) as f:
            orig_annotation = json.load(f)
        cap = cv2.VideoCapture(os.path.join(base_path, date, clip, 'color.mp4'))
        for i in range(0, int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), 3):
            cap.set(1, i)
            ret, frame = cap.read()
            masked_frame = (frame * mask).astype(np.uint8)
            frame_filename = f'{date}__{clip}__{i}.jpeg'
            cv2.imwrite(os.path.join(out_detection_path, frame_filename), masked_frame)

NotADirectoryError: [WinError 267] Неверно задано имя папки: 'annotated/mask.jpg'

In [4]:
with open('recognition_annotations.json') as f:
            recognition_annotations = json.load(f)

In [5]:
recognition_annotations

[{'frameNumber': 0,
  'bbox': {'x': 196, 'y': 299, 'width': 240, 'height': 121},
  'isGroundTruth': True,
  'visible': True,
  'behaviour': 'investigating',
  'id': '3',
  'recognition_filename': '2019_11_05__000002__0__3.jpeg'},
 {'frameNumber': 0,
  'bbox': {'x': 462, 'y': 438, 'width': 216, 'height': 88},
  'isGroundTruth': True,
  'visible': True,
  'behaviour': 'standing',
  'id': '4',
  'recognition_filename': '2019_11_05__000002__0__4.jpeg'},
 {'frameNumber': 0,
  'bbox': {'x': 927, 'y': 402, 'width': 131, 'height': 165},
  'isGroundTruth': True,
  'visible': True,
  'behaviour': 'walk',
  'id': '6',
  'recognition_filename': '2019_11_05__000002__0__6.jpeg'},
 {'frameNumber': 0,
  'bbox': {'x': 442, 'y': 145, 'width': 141, 'height': 235},
  'isGroundTruth': True,
  'visible': True,
  'behaviour': 'investigating',
  'id': '7',
  'recognition_filename': '2019_11_05__000002__0__7.jpeg'},
 {'frameNumber': 0,
  'bbox': {'x': 570, 'y': 134, 'width': 173, 'height': 231},
  'isGroundTru

In [21]:
images_per_action = {}
for annot in recognition_annotations:
    action = annot['behaviour']
    images = images_per_action.get(action, [])
    if len(images) < 10:
        images.append(cv2.resize(cv2.imread(os.path.join(out_recognition_path, annot['recognition_filename'])), (225,225)))
        images_per_action[action] = images
collage = []
for action, images in images_per_action.items():
    print()
    if len(images)< 10:
        images.append(np.zeros((225, 225*(10 - len(images)), 3)))
    collage_row = np.hstack(images)
    cv2.putText(collage_row,action, (10,80), cv2.FONT_HERSHEY_SIMPLEX, 2, 255, 10)
    collage.append(collage_row)
collage = np.vstack(collage)

In [23]:
cv2.imwrite('classes_collage.jpeg', collage.astype(np.uint8))

-1

In [ ]:
print(collage[0:225, 0:225,:])